# Semantic Tool Discovery: Reducing Tool Selection Hallucinations

Based on: [Internal Representations as Indicators of Hallucinations in Agent Tool Selection](https://arxiv.org/pdf/2601.05214)

## The Problem

When an AI agent has access to many tools with similar names and descriptions (e.g., `search_hotels`, `search_hotel_reviews`, `get_hotel_details`, `get_hotel_pricing`), it often picks the wrong one. The paper calls this **tool selection hallucination** — the agent confidently chooses a tool that doesn't match the user's intent.

This gets worse as the number of tools grows. With 5 tools, the agent rarely makes mistakes. With 30+, confusion between similar tools becomes a real problem.

## The Technique: Semantic Pre-filtering with FAISS

Instead of giving the agent all 31 tools at once, we use a two-step approach:

**Step 1 — Index tools by meaning**: We take each tool's name + docstring (e.g., `"get_hotel_pricing: Get room pricing for a hotel."`) and convert it into a vector embedding using `SentenceTransformer('all-MiniLM-L6-v2')`. These embeddings are stored in a FAISS index for fast similarity search.

**Step 2 — Pre-filter before the agent sees them**: When a user query arrives, we embed it with the same model, search the FAISS index for the top-3 closest tools, and create the agent with **only those 3 tools**. The agent never sees the other 28.

```
User query → Embed → FAISS search → Top-3 tools → Agent(tools=top3)
```

## Why This Works

- **91% fewer choices**: 3 tools instead of 31 means less room for confusion
- **No generic traps**: Ambiguous tools like `search()` or `get_info()` are filtered out unless truly relevant
- **Semantic, not keyword**: "How much does Hotel Marriott cost?" matches `get_hotel_pricing` by meaning, not just by the word "hotel"

## What We Test

We run 13 queries against both approaches and compare:
- **Traditional**: Agent gets all 31 tools
- **Semantic**: Agent gets only the top-3 tools per query

Each query has a known correct tool (ground truth). We measure how often each approach picks the right one.

In [ ]:
!pip install requirements.txt -r

## Setup

In [ ]:
import sys
import io
import re

from strands import Agent
from enhanced_tools import ALL_TOOLS
from registry import build_index, search_tools, get_scores

print(f"✅ Loaded {len(ALL_TOOLS)} tools")

## Build Semantic Index

For each tool, we concatenate `name: docstring` and encode it with `all-MiniLM-L6-v2` (384-dim vectors). FAISS stores these vectors and performs L2 nearest-neighbor search. The score shown is `1/(1+distance)` — closer to 1.0 means better match.

In [ ]:
build_index(ALL_TOOLS)

# Test semantic search
test_query = "Find hotels in Spain"
print(f"\n🔍 Query: '{test_query}'")
print("\nTop 5 semantically similar tools:")
scores = get_scores(test_query, top_k=5)
for i, s in enumerate(scores, 1):
    print(f"{i}. {s['name']:30} (score: {s['score']:.3f})")

## Test Queries (Ground Truth)

In [ ]:
TESTS = [
    # Original tests
    ("What's the weather in Paris?", "get_weather"),
    ("Find flights from NYC to London", "search_flights"),
    ("Book a hotel in Rome for John", "book_hotel"),
    ("Check flight status for AA123", "get_flight_status"),
    ("How much does Hotel Marriott cost?", "get_hotel_pricing"),
    ("Cancel my reservation", "cancel"),
    ("Search hotels in Barcelona", "search_hotels"),
    ("Flight prices to Tokyo", "search_flight_prices"),
    
    # Enhanced tests with real data
    ("Show me top-rated hotels", "get_top_hotels"),
    ("Find hotels in France with rating above 9", "search_real_hotels"),
    ("Convert 500 USD to EUR", "get_currency_exchange"),
    ("Do I need a visa for Spain from USA?", "get_travel_documents"),
    ("Check availability for Hilton on 2026-03-15 to 2026-03-18", "check_hotel_availability_dates"),
]

print(f"📋 Test suite: {len(TESTS)} queries")
print(f"📊 Tool pool: {len(ALL_TOOLS)} tools")

## Helper Function

In [ ]:
def run_and_capture(agent, query):
    """Run agent and capture tool calls"""
    old_stdout = sys.stdout
    sys.stdout = io.StringIO()
    
    try:
        result = agent(query)
        output = sys.stdout.getvalue()
    finally:
        sys.stdout = old_stdout
    
    tools = re.findall(r'Tool #\d+: (\w+)', output)
    return tools

---
## Test 1: Traditional Approach (All Tools)

The agent receives all 31 tools in its system prompt. It must figure out which one to call based on the tool names and docstrings alone. With 7 generic tools (`search`, `check`, `get_details`, `get_status`, `get_info`, `book`, `cancel`) competing against specific ones, the model is likely to pick a generic or similar-but-wrong tool.

In [ ]:
MODEL = "us.anthropic.claude-3-haiku-20240307-v1:0"
PROMPT = "You are a travel assistant. Use the correct tool to answer questions."

print("="*70)
print(f"TEST 1: TRADITIONAL - {len(ALL_TOOLS)} tools")
print("="*70)

trad_results = []
trad_correct = 0

for query, expected in TESTS:
    agent = Agent(tools=ALL_TOOLS, system_prompt=PROMPT, model=MODEL)
    tools = run_and_capture(agent, query)
    ok = expected in tools
    trad_correct += ok
    trad_results.append({'query': query, 'expected': expected, 'actual': tools, 'correct': ok})
    print(f"{'✅' if ok else '❌'} {query[:45]:45} -> {tools[:2] if tools else 'NO TOOL'}")

print(f"\n📊 Traditional: {trad_correct}/{len(TESTS)} ({100*trad_correct/len(TESTS):.1f}%)")

---
## Test 2: Semantic Approach (Pre-filtered Tools)

This test uses `registry.py` to dynamically select which tools the agent receives. Here's how it works:

### How the Tool Registry Works (`registry.py`)

**1. Building the vector index — `build_index(tools)`**

Called once at startup. For each tool, it concatenates `name: docstring` (e.g., `"get_hotel_pricing: Get room pricing for a hotel."`) and encodes it into a 384-dimensional vector using `SentenceTransformer('all-MiniLM-L6-v2')`. All vectors are added to a FAISS `IndexFlatL2` index (L2 = Euclidean distance). The index lives in memory — if it doesn't exist yet, `search_tools()` will fail, so `build_index()` must run first.

**2. Querying — `search_tools(query, top_k=3)`**

When a user query arrives, it's embedded with the same model and searched against the FAISS index. FAISS returns the `top_k` nearest tool vectors by distance. The function returns the actual tool callables (not just names), ready to pass directly to `Agent(tools=...)`.

**3. Agent creation — per-query tool injection**

For each query, a new `Agent` is created with **only the top-3 tools**. The other 28 tools are never passed to the agent — they're not removed, they simply never enter the agent's context. This is the key difference: the agent's tool selection space shrinks from 31 to 3.

**4. Why `all-MiniLM-L6-v2`?**

It's a lightweight sentence embedding model (22M params, 384 dims) optimized for semantic similarity. It runs fast on CPU, which matters here since we're encoding short strings (tool descriptions), not documents. Larger models would be overkill for matching a query like "How much for a room?" to `"get_hotel_pricing: Get room pricing for a hotel."`

```
build_index(ALL_TOOLS)          # Once: encode 31 tool descriptions → FAISS
selected = search_tools(query)  # Per query: embed query → top-3 tools
Agent(tools=selected)           # Agent only sees 3 tools, not 31
```

In [ ]:
print("="*70)
print("TEST 2: SEMANTIC - 3 tools per query")
print("="*70)

sem_results = []
sem_correct = 0

for query, expected in TESTS:
    selected = search_tools(query, top_k=3)
    selected_names = [t.__name__ for t in selected]
    
    agent = Agent(tools=selected, system_prompt=PROMPT, model=MODEL)
    tools = run_and_capture(agent, query)
    ok = expected in tools
    sem_correct += ok
    sem_results.append({'query': query, 'expected': expected, 'selected': selected_names, 'actual': tools, 'correct': ok})
    print(f"{'✅' if ok else '❌'} {query[:45]:45} -> {tools[:2] if tools else 'NO TOOL'}")
    print(f"   Available: {selected_names}")

print(f"\n📊 Semantic: {sem_correct}/{len(TESTS)} ({100*sem_correct/len(TESTS):.1f}%)")

---
## Test 3: Semantic Approach with Memory (Single Agent)

Tests 1 and 2 create a **new agent per query**. That works for benchmarking, but in production you lose conversation history — the agent can't reference previous answers or maintain context across turns.

### The Problem

Strands agents store conversation history in `agent.messages`. When you do `Agent(tools=selected)` for each query, you get a fresh agent with an empty message list. Multi-turn conversations break.

### The Solution: `swap_tools(agent, new_tools)`

Instead of recreating the agent, we manipulate its `tool_registry` directly:

1. Clear `agent.tool_registry.registry` and `agent.tool_registry.dynamic_tools`
2. Re-register only the tools returned by `search_tools(query)`
3. Call the same agent — `agent.messages` is untouched

This works because Strands calls `tool_registry.get_all_tools_config()` at each event loop cycle, so runtime changes to the registry are picked up on the next `agent()` call.

```
agent = Agent(tools=initial_tools)   # Create once
for query in queries:
    selected = search_tools(query)    # FAISS top-3
    swap_tools(agent, selected)       # Swap registry, keep memory
    agent(query)                      # Conversation history preserved
```

In [ ]:
from registry import swap_tools

print("="*70)
print("TEST 3: SEMANTIC + MEMORY - 3 tools per query, single agent")
print("="*70)

# Create ONE agent with initial tools (first query's tools)
initial_tools = search_tools(TESTS[0][0], top_k=3)
memory_agent = Agent(tools=initial_tools, system_prompt=PROMPT, model=MODEL)

mem_results = []
mem_correct = 0

for i, (query, expected) in enumerate(TESTS):
    # Swap tools for this query (memory preserved)
    selected = search_tools(query, top_k=3)
    selected_names = [t.__name__ for t in selected]
    swap_tools(memory_agent, selected)
    
    tools = run_and_capture(memory_agent, query)
    ok = expected in tools
    mem_correct += ok
    mem_results.append({'query': query, 'expected': expected, 'selected': selected_names, 'actual': tools, 'correct': ok})
    print(f"{'✅' if ok else '❌'} {query[:45]:45} -> {tools[:2] if tools else 'NO TOOL'}")
    print(f"   Available: {selected_names} | Memory: {len(memory_agent.messages)} msgs")

print(f"\n📊 Semantic+Memory: {mem_correct}/{len(TESTS)} ({100*mem_correct/len(TESTS):.1f}%)")
print(f"💬 Total conversation messages: {len(memory_agent.messages)}")

---
## Ground Truth Verification

We compare each agent's tool selection against the known correct tool. Any mismatch is a **tool selection hallucination**. For semantic errors, we also check whether the correct tool was even in the top-3 — if it wasn't, the error is in the embedding search, not the agent.

In [ ]:
print("="*70)
print("GROUND TRUTH VERIFICATION")
print("="*70)

print(f"\n📊 Results:")
print(f"   Traditional:      {trad_correct}/{len(TESTS)} ({100*trad_correct/len(TESTS):.1f}%)")
print(f"   Semantic:         {sem_correct}/{len(TESTS)} ({100*sem_correct/len(TESTS):.1f}%)")
print(f"   Semantic+Memory:  {mem_correct}/{len(TESTS)} ({100*mem_correct/len(TESTS):.1f}%)")
print(f"   Improvement:      +{sem_correct - trad_correct} queries (+{100*(sem_correct-trad_correct)/len(TESTS):.1f}%)")

print(f"\n🔍 Traditional Errors:")
for r in trad_results:
    if not r['correct']:
        print(f"   ❌ '{r['query'][:50]}'")
        print(f"      Expected: {r['expected']}, Got: {r['actual']}")

print(f"\n🔍 Semantic Errors:")
for r in sem_results:
    if not r['correct']:
        print(f"   ❌ '{r['query'][:50]}'")
        print(f"      Expected: {r['expected']}, Got: {r['actual']}")
        print(f"      Available: {r['selected']}")
        if r['expected'] not in r['selected']:
            print(f"      ⚠️  Correct tool NOT in top-3!")

print(f"\n🔍 Semantic+Memory Errors:")
for r in mem_results:
    if not r['correct']:
        print(f"   ❌ '{r['query'][:50]}'")
        print(f"      Expected: {r['expected']}, Got: {r['actual']}")
        print(f"      Available: {r['selected']}")

---
## Summary

### Key Findings

| Approach | Tools Available | Memory | Accuracy | Hallucination Rate |
|----------|----------------|--------|----------|--------------------|
| Traditional | 33 tools | ❌ New agent/query | X/13 (Y%) | (100-Y)% |
| Semantic | 3 tools/query | ❌ New agent/query | X/13 (Y%) | (100-Y)% |
| Semantic+Memory | 3 tools/query | ✅ Single agent | X/13 (Y%) | (100-Y)% |

### Why Semantic Tool Discovery Reduces Hallucinations

1. **Reduced Search Space**: 3 tools instead of 33 → 91% reduction
2. **Semantic Relevance**: Pre-filtered by meaning
3. **Clearer Context**: Better understanding of each tool
4. **Less Confusion**: Similar tools are separated
5. **Memory Compatible**: `swap_tools()` preserves conversation history while still pre-filtering

### Limitations

- Semantic search quality depends on docstring quality
- If correct tool not in top-3, agent will fail
- Requires upfront embedding computation

### References

- [Internal Representations as Indicators of Hallucinations in Agent Tool Selection](https://arxiv.org/pdf/2601.05214)